In [ ]:
!wget https://s3.amazonaws.com/ifashionist-dataset/images/train2020.zip
!wget https://s3.amazonaws.com/ifashionist-dataset/images/val_test2020.zip
!wget https://s3.amazonaws.com/ifashionist-dataset/annotations/instances_attributes_train2020.json
!wget https://s3.amazonaws.com/ifashionist-dataset/annotations/instances_attributes_val2020.json
!wget https://s3.amazonaws.com/ifashionist-dataset/annotations/info_test2020.json

In [ ]:
import pandas as pd
import json
datatrain=json.load(open('instances_attributes_train2020.json'))
dataval=json.load(open('instances_attributes_val2020.json'))

In [ ]:
annot_train=pd.DataFrame(datatrain['annotations'])
annot_val=pd.DataFrame(dataval['annotations'])
annot_train.head()

In [ ]:
image_attri_train=pd.DataFrame(datatrain['images'])
image_attri_val=pd.DataFrame(dataval['images'])
image_attri_train.head()

In [ ]:
!unzip -xq train2020.zip
!unzip -xq val_test2020.zip
!mv test val

In [ ]:
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/val

In [ ]:
import os
clss={0:'shirt',1:'top',2:'sweater',3:'cardigan',4:'jacket',
       5:'vest',6:'pants',7:'shorts',8:'skirt',9:'coat',10:'dress',11:'jumpsuit'}
for i in range(0,12):
    os.mkdir('dataset/train/{}/'.format(clss[i]))
    os.mkdir('dataset/val/{}/'.format(clss[i]))

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
def create_txt(annot,attrip,typ,path):
  id=1
  for i in tqdm(annot['image_id'].unique()):
    image_annotaions=annot[annot['image_id']==i]
    attriputes=attrip[attrip['id']==i]
    img_name=attrip[attrip['id']==i]['file_name']
    img_name=img_name[img_name.keys()[0]]
    img=cv2.imread('{}/{}'.format(typ,img_name))
    lines=[]
    for cat,seg in zip(image_annotaions.category_id,image_annotaions.segmentation):
      if cat not in range(0,12):
        continue
      pts=[]
      try:
        for poly in seg:
          pts.extend([[poly[pt],poly[pt+1]] for pt in range(0,len(poly),2)])
        mask = np.zeros(img.shape[:2], np.uint8)
        cv2.drawContours(mask, [np.array(pts)], -1, (255, 255, 255), -1, cv2.LINE_AA)
        dst = cv2.bitwise_and(img, img, mask=mask)
        cv2.imwrite('{}/{}/{}.jpg'.format(path,clss[cat],id),dst)
        if id==1:
          cv2.imwrite('{}{}.jpg'.format(path,id),dst)
        id+=1
      except:
        pass
  print(len(os.listdir(path)))
create_txt(annot_train,image_attri_train,'train','dataset/train')
create_txt(annot_val,image_attri_val,'val','dataset/val')

In [ ]:
!pip install ultralytics

In [ ]:
for i in os.listdir('dataset/train'):
    print(i , len(os.listdir('dataset/train/{}'.format(i))))

In [ ]:
from ultralytics import YOLO
model=YOLO('yolov8n-cls.pt')
model.train(data='dataset',epochs=10,imgsz=640)